Langchain快速入门

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_API_BASE")


In [3]:
print(api_key)
print(base_url)

sk-acUzWFHVvFTIaINs4fF7Bc2b30F24b45AdC97dAa6aAe8cEa
https://dd-ai-api.eastmoney.com/v1


In [13]:
# 配置deepseek模型
llm = ChatOpenAI(
    model="deepseek-r1",  # 或其他 DeepSeek 模型名称
    openai_api_key=api_key,
    openai_api_base=base_url,  # DeepSeek 的 API 地址
    # temperature=0.7
)

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ('system', '你是世界级的专家'),
    ("user", "{input}")
])

In [7]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [14]:
chain = prompt | llm | output_parser

In [15]:
result = chain.invoke({'input':"给我一份初学者学习langchain框架的建议，大概两三百个字"})

In [11]:
result1 = llm.invoke("给我一份初学者学习langchain框架的建议，大概两三百个字")

In [12]:
print(result)



学习LangChain建议从核心概念入手，逐步构建实战能力。作为初学者，先理解其三大核心：**提示工程**（Prompt Engineering）、**链式调用**（Chaining）和**模型集成**（Model Integration）。官方文档（https://js.langchain.com）是首选资料，配合《LangChain for LLM Application Development》课程（免费）建立系统认知。

实践路径建议分三步走：  
1️⃣ **基础搭建**：用HuggingFace或OpenAI API跑通第一个问答链（ RetrievalQA），熟悉DocumentLoader、TextSplitter等数据处理模块。  
2️⃣ **场景实战**：尝试构建聊天机器人/智能Agent，重点掌握Memory模块（如ConversationBufferMemory）和Chain组合逻辑（LLMChain + Database Chain）。  
3️⃣ **进阶优化**：学习自定义Chain（Custom Chain）和Callback处理器，通过LangSmith进行链路调试，用LangServe部署服务。

推荐搭配学习栈：Python基础→LLM基础概念→Prompt Engineering指南→LangChain Cookbook示例。每周完成1个GitHub示例复现（如PDF解析器），参与Discord社区讨论解决报错问题。注意关注v0.1+版本的模块化重构，避免陷入历史API陷阱。保持代码与文档同步迭代，逐步构建工程化思维。


In [11]:
print(result1.mete)

NameError: name 'result1' is not defined

提示词工程实践

创建提示词模板 prompt template

In [14]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("给我讲一个关于{content}的{adjective}{syj}")

res = prompt_template.format(adjective = '冷', content = '猴子', syj = '名字')
print(res)

给我讲一个关于猴子的冷名字


聊天消息提示词模板 chat prompt template

In [15]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ('system','你是一位人工智能住手，你的名字是{name}'),
    ('human','你好'),
    ('ai', '我很好，谢谢！'),
    ('human', '{user_input}')
])

message = chat_template.format_messages(name = '三叶', user_input = '你叫什么名字？')
print(message)

[SystemMessage(content='你是一位人工智能住手，你的名字是三叶', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='我很好，谢谢！', additional_kwargs={}, response_metadata={}), HumanMessage(content='你叫什么名字？', additional_kwargs={}, response_metadata={})]


In [16]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(
        content = (
            '你是一个乐于助人的住手，可以润色内容，使其看起来更简单易懂。'
        )
    ),
    HumanMessagePromptTemplate.from_template("{text}")
])

message = chat_template.format_messages(text = '我不喜欢吃好吃的东西')
print(message)

[SystemMessage(content='你是一个乐于助人的住手，可以润色内容，使其看起来更简单易懂。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我不喜欢吃好吃的东西', additional_kwargs={}, response_metadata={})]


MessagesPlaceholder \
在特定位置增加消息列表

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ('system', "You are a helpful assistant"),
    MessagesPlaceholder('msgs')
])
prompt_template.invoke({'msgs':[HumanMessage(content='hi')]})

prompt_template

ChatPromptTemplate(input_variables=['msgs'], input_types={'msgs': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.fu

提示词追加示例 Few-shot prompt templates

使用示例集

In [31]:
# 创建示例集
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts import PromptTemplate

examples = [
  {
    "question": "谁的寿命更长，穆罕默德·阿里还是艾伦·图灵？",
    "answer":
"""
这里需要跟进问题吗：是的。
跟进：穆罕默德·阿里去世时多大？
中间答案：穆罕默德·阿里去世时74岁。
跟进：艾伦·图灵去世时多大？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里
"""
  },
  {
    "question": "craigslist的创始人是什么时候出生的？",
    "answer":
"""
这里需要跟进问题吗：是的。
跟进：craigslist的创始人是谁？
中间答案：craigslist由Craig Newmark创立。
跟进：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark于1952年12月6日出生。
所以最终答案是：1952年12月6日
"""
  },
  {
    "question": "乔治·华盛顿的祖父母中的母亲是谁？",
    "answer":
"""
这里需要跟进问题吗：是的。
跟进：乔治·华盛顿的母亲是谁？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
跟进：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball
"""
  },
  {
    "question": "《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？",
    "answer":
"""
这里需要跟进问题吗：是的。
跟进：《大白鲨》的导演是谁？
中间答案：《大白鲨》的导演是Steven Spielberg。
跟进：Steven Spielberg来自哪里？
中间答案：美国。
跟进：《皇家赌场》的导演是谁？
中间答案：《皇家赌场》的导演是Martin Campbell。
跟进：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是
"""
  }
]

In [35]:
# 创建小样本示例的格式化对象

eaxmple_prompt = PromptTemplate(input_variables = ['question', 'answer'], template = '问题：{question}\\n{answer}')

print(eaxmple_prompt.format(**examples[0]))

问题：谁的寿命更长，穆罕默德·阿里还是艾伦·图灵？\n
这里需要跟进问题吗：是的。
跟进：穆罕默德·阿里去世时多大？
中间答案：穆罕默德·阿里去世时74岁。
跟进：艾伦·图灵去世时多大？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里



In [36]:
#将示例和格式化程序提供给 FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = eaxmple_prompt,
    suffix = '问题：{input}',
    input_variables = ['input'],
)

print(prompt.format(input = '乔治·华盛顿的父亲是谁'))

问题：谁的寿命更长，穆罕默德·阿里还是艾伦·图灵？\n
这里需要跟进问题吗：是的。
跟进：穆罕默德·阿里去世时多大？
中间答案：穆罕默德·阿里去世时74岁。
跟进：艾伦·图灵去世时多大？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里


问题：craigslist的创始人是什么时候出生的？\n
这里需要跟进问题吗：是的。
跟进：craigslist的创始人是谁？
中间答案：craigslist由Craig Newmark创立。
跟进：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark于1952年12月6日出生。
所以最终答案是：1952年12月6日


问题：乔治·华盛顿的祖父母中的母亲是谁？\n
这里需要跟进问题吗：是的。
跟进：乔治·华盛顿的母亲是谁？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
跟进：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball


问题：《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？\n
这里需要跟进问题吗：是的。
跟进：《大白鲨》的导演是谁？
中间答案：《大白鲨》的导演是Steven Spielberg。
跟进：Steven Spielberg来自哪里？
中间答案：美国。
跟进：《皇家赌场》的导演是谁？
中间答案：《皇家赌场》的导演是Martin Campbell。
跟进：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是


问题：乔治·华盛顿的父亲是谁


使用示例选择器\
这里重用前一部分中的示例集和提示词模板(prompt template)。但是，不会将示例直接提供给FewShotPromptTemplate对象，把全部示例插入到提示词中，而是将它们提供给一个ExampleSelector对象，插入部分示例。

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="sentence-transformers/all-MiniLM-L6-v2")

In [47]:
# 将示例提供给ExampleSelector
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)


examples_selector = SemanticSimilarityExampleSelector.from_examples(
    # 这是可供选择的示例列表。
    examples,
    # 这是用于生成嵌入的嵌入类，该嵌入用于衡量语义相似性。
    embeddings,
    # 这是用于存储嵌入和执行相似性搜索的VectorStore类。
    Chroma,
    # 这是要生成的示例数。
    k=1
)

question = '乔治·华盛顿的父亲是谁？'
selected_examples = examples_selector.select_examples({'question':question})
print({question})
for example in selected_examples:
    print('\n')
    for k, v in example.items():
        print(f'{k}: {v}')

{'乔治·华盛顿的父亲是谁？'}


answer: 
这里需要跟进问题吗：是的。
跟进：乔治·华盛顿的母亲是谁？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
跟进：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball

question: 乔治·华盛顿的祖父母中的母亲是谁？
